# Exercise 1 - Linear Regression

The first exercise is about linear models.
The given data set contains prices and other attributes of approximately 54,000 diamonds. You should fit a linear model to predict the price of a diamond, given its attributes.

This exercise is meant to get you started with the tool stack. Besides numpy and matplotlib we use the following python packages:

- [pandas](https://pandas.pydata.org/)
- [sklearn](http://scikit-learn.org/)

If you are unfamiliar with them, follow the documentation links. 

In the event of a persistent problem, do not hesitate to contact the course instructor under

- paul.kahlmeyer@uni-jena.de

### Submission
- Deadline of submission:
        19.04.23 23:59
- Submission on [moodle page](https://moodle.uni-jena.de/course/view.php?id=43681)


### Help
In case you cannot solve a task, you can use the saved values within the `help` directory:
- Load arrays with [Numpy](https://numpy.org/doc/stable/reference/generated/numpy.load.html)
```
np.load('help/array_name.npy')
```
- Load functions with [Dill](https://dill.readthedocs.io/en/latest/dill.html)
```
import dill
with open('help/some_func.pkl', 'rb') as f:
    func = dill.load(f)
```

to continue working on the other tasks.

## Preprocessing

We use the same notation as in the lecture.
- $m$... Number of datapoints
- $n$... Number of features

### Dataset 

As a dataset, we use the [diamond dataset](https://www.kaggle.com/shivam2503/diamonds).

Each element in this dataset represents a diamond and has the following features:

- price: price in US dollars (326.0 - 18823.0)
- carat: weight of the diamond (0.2 - 5.01)
- cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
- color: diamond colour, from J (worst) to D (best)
- clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
- x: length in mm (0-10.74)
- y: width in mm (0-58.9)
- z: depth in mm (0-31.8)
- depth: total depth percentage = 2 * z / (x + y) (43-79)
- table: width of top of diamond relative to widest point (43-95)

The dataset is stored under `diamonds.csv`.

### Task 1
Import the data from the file using [pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) and examine it.

Determine the following:

* The number of data points
* The column names
* The data types for each column

In [1]:
# TODO: load data
import pandas as pd
df = pd.read_csv("diamonds.csv")

# TODO: determine number of datapoints
print("dataset size", df.size)
print(df.info)

# TODO: determine column names
col_names = df.columns
print(col_names)

# TODO: determine datatypes of columns
dtypes = df.dtypes
print(dtypes)

df

dataset size 539400
<bound method DataFrame.info of        carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 column

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


### Task 2

Since there are discrete variables and we do not yet know how to include them into our regression model, remove them. Additionally, verify that there are no missing values in our dataset.

Hint: there are multiple ways to [check](https://towardsdatascience.com/how-to-check-for-missing-values-in-pandas-d2749e45a345) for missing values

In [2]:
import numpy as np

# TODO: remove discrete variables
if "cut" in df.columns:
    df.drop("cut", axis=1)
    df.drop("color", axis=1)
    df.drop("clarity", axis=1)

# check for missing values
assert not np.isnan(df.values).any(), "There are missing values!"

As discussed in the lecture, we should **standardize** the data, to make different scales comparable.

Standardization is defined for each feature $x_i$:

\begin{align}
\hat{x}_i = \cfrac{x_i-\mu_x}{\sigma_x}\,,
\end{align}
where $\mu_x$ and $\sigma_x$ are the empirical [mean](https://en.wikipedia.org/wiki/Mean) and [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) of the feature $x$.

### Task 3

Convert the pandas dataframe to a numpy array and calculate the standardized data matrix $X$.

In [146]:
def pm(title: str, a: np.ndarray) -> None:
    print(f"\n{title} {a.shape}")
    print(a)

In [170]:
# TODO: calculate standardized data matrix X
X_orig = np.array(df.loc[:, df.columns != "price"])
rows = X_orig.shape[0]

# empirical mean / Erwartungswert
M = np.einsum('ij->j', X_orig) / rows
pm("empirical mean μ", M)

pm("features X_original", X_orig)

S = X_orig - M
S *= S
S = np.sum(S, axis=0)
S /= rows
pm("standard deviation σ", S)

X = (X_orig - M) / S
pm("transformed features X", X)


empirical mean μ (6,)
[ 0.79793975 61.74940489 57.45718391  5.73115721  5.73452595  3.53873378]

features X_original (53940, 6)
[[ 0.23 61.5  55.    3.95  3.98  2.43]
 [ 0.21 59.8  61.    3.89  3.84  2.31]
 [ 0.23 56.9  65.    4.05  4.07  2.31]
 ...
 [ 0.7  62.8  60.    5.66  5.68  3.56]
 [ 0.86 61.   58.    6.15  6.12  3.74]
 [ 0.75 62.2  55.    5.83  5.87  3.64]]

standard deviation σ (6,)
[0.22468249 2.05236579 4.99285551 1.25832384 1.30444743 0.49800163]

transformed features X (53940, 6)
[[-2.52774365 -0.12152068 -0.49214    -1.41549985 -1.34503385 -2.22636576]
 [-2.61675815 -0.94983307  0.70957713 -1.46318233 -1.45235899 -2.46732883]
 [-2.52774365 -2.36283654  1.51072189 -1.33602905 -1.27603912 -2.46732883]
 ...
 [-0.43590289  0.51189467  0.50929094 -0.0565492  -0.04180004  0.04270312]
 [ 0.27621312 -0.36514197  0.10871857  0.33285771  0.29550754  0.40414772]
 [-0.21336664  0.21954912 -0.49214     0.07855115  0.1038555   0.20334516]]


In [168]:
# test equivalence of operations
for i in np.arange(X.shape[0]):
    for j in np.arange(X.shape[1]):
        assert X[i,j] == (X_orig[i,j] - M[j]) / S[j]

### Task 4

Scikit learn has an [implementation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) of this preprocessing.

Use it to create a second standardized data matrix and compare this result with your result from Task 3.

In [174]:
# TODO: compare to sklearn result
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
diamond_scaler = scaler.fit(X_orig)


pm("Mean old", M)
pm("Mean Skl", diamond_scaler.mean_)
pm("Mean dif", M - diamond_scaler.mean_)

pm("Std old", S)
pm("Std Skl", diamond_scaler.var_)
pm("Std dif", S - diamond_scaler.var_)

X2 = diamond_scaler.transform(X_orig)

pm("sklearn scaler", X2)
pm("old scale", X)

Deviation = X - X2
pm("Deviation between scalers", Deviation)

Deviation2 = (X/2) - X2
pm("Deviation between scalers with 1/2 * X", Deviation2)

# TODO: I do not understand why the results differ!


Mean old (6,)
[ 0.79793975 61.74940489 57.45718391  5.73115721  5.73452595  3.53873378]

Mean Skl (6,)
[ 0.79793975 61.74940489 57.45718391  5.73115721  5.73452595  3.53873378]

Mean dif (6,)
[ 3.10862447e-15  0.00000000e+00  0.00000000e+00 -4.44089210e-15
 -7.10542736e-15  8.88178420e-16]

Std old (6,)
[0.22468249 2.05236579 4.99285551 1.25832384 1.30444743 0.49800163]

Std Skl (6,)
[0.22468249 2.05236579 4.99285551 1.25832384 1.30444743 0.49800163]

Std dif (6,)
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.66533454e-16]

sklearn scaler (53940, 6)
[[-1.19816781 -0.17409151 -1.09967199 -1.58783745 -1.53619556 -1.57112919]
 [-1.24036129 -1.36073849  1.58552871 -1.64132529 -1.65877419 -1.74117497]
 [-1.19816781 -3.38501862  3.37566251 -1.49869105 -1.45739502 -1.74117497]
 ...
 [-0.20662095  0.73334442  1.13799526 -0.06343409 -0.04774083  0.03013526]
 [ 0.13092691 -0.52310533  0.24292836  0.37338325  0.33750627  0.28520393]
 [-0.10113725  0.31452784 -1.0

## Inspecting the Data

Visualizing correlation in your data often helps to build intuition and get a feeling of the deeper mojo in the set.

Here we want to use the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) as a measure for correlation between two variables.

Let $x$ and $y$ be two variables of the unstandardized dataset (e.g. `carat` and `price`). The empirical Pearson correlation coefficient between $x$ and $y$ is defined as 

\begin{align}
r_{xy} = \cfrac{\sum_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^n(x_i-\bar{x})^2}\sqrt{\sum_{i=1}^n(y_i-\bar{y})^2}}\,,
\end{align}
where $\bar{x}$ and $\bar{y}$ are the respective empirical means.

### Task 5

How does this definition translate to our standardized data matrix $X$?

Calculate the pairwise correlation matrix for our dataset. 

Visualize this correlation matrix and label the rows/columns.

In [5]:
# TODO: calculate correlation matrix

# TODO: visualize correlation matrix

## Linear Regression

Our goal in this exercise will be to predict the `price` of a diamond based on some of its other features.

We will use linear regression, that is we assume the `price` (=$y$) depends linearly on the other features (=$\mathbf{x}$):
\begin{align}
y = \theta^T \mathbf{x} + \varepsilon
\end{align}
where $\varepsilon$ is standard normal distributed noise.

In `Linear_Regression_Script.pdf` you find how the maximum likelihood estimate $\hat\theta$ is calculated.

### Task 6

Implement a `LinReg` class that uses maximum likelihood estimation. Add the possibility to use [Ridge Regression](https://en.wikipedia.org/wiki/Ridge_regression).

In [6]:
class LinReg():
    
    def __init__(self, c:int = 0):
        '''
        Class for linear regression.
        
        @Params:
            c... parameter for ridge regression
        '''
        # TODO: create attributes: c, theta 
        pass
    
    def fit(self, X:np.ndarray, Y:np.ndarray) -> None:
        '''
        Learns the parameters for a linear regression task.
        
        @Params:
            X... n x m matrix
            Y... vector of length m
        '''
        
        # TODO: estimate theta
        pass
        
    def predict(self, X:np.ndarray) -> np.ndarray:
        '''
        Using learned parameters, predicts output for given X.
        
        @Params:
            X... n x m matrix
            Y... vector of length m
        '''
        
        # TODO: predict labels
        pass
    

### Task 7

First we want to predict the `price` of a diamond solely from the variable `carat`. 
Make a scatter plot of `carat` vs `price` using matplotlib. Label the axes and give the plot a title.

Use the standardized dataset.

In [7]:
# TODO: display data in scatter plot

### Task 8

Set up the design matrix and use your class to estimate $\theta$ on the dataset.
Note, that the design matrix does **not** need the vector of ones, since we standardized the dataset.

Plot the regression line defined by $\theta$.

In [8]:
# TODO: build design matrix, y

# TODO: use Linear Regression

# TODO: plot data + regression line

### Task 9

You can find an implementation of this method in the python module [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Use it and compare your result for the estimation of $\theta$.

**Important:** scikit learn needs the design matrix as a $m \times n$ matrix (datapoints as rows).

In [9]:
# TODO: use scikit learn to estimate theta

# TODO: compare results

### Task 10

Now predict the `price` from the variables `carat`, `depth`, `table`, `x`, `y`, `z`.

- Estimate $\theta$ with your class
- Estimate $\theta$ with scikit learn
- Compare both estimations

In [10]:
# TODO: build X, Y

# TODO: estimate theta

# TODO: estimate theta using scikit-learn + compare